# Factor Risk Reports

Factor risk reports run historical factor risk analyses for your portfolio or basket over a specified date range by leveraging a factor risk model of your choice.

## Step 1: Authenticate and Initialize Your Session

First you will import the necessary modules and add your client id and client secret.

In [ ]:
import datetime as dt
from time import sleep

import pandas as pd

from gs_quant.markets.report import FactorRiskReport
from gs_quant.models.risk_model import FactorRiskModel
from gs_quant.session import GsSession, Environment

client = None
secret = None
scopes = None

## External users must fill in their client ID and secret below and comment out the line below
# client = 'ENTER CLIENT ID'
# secret = 'ENTER CLIENT SECRET'

GsSession.use(
    Environment.PROD,
    client_id=client,
    client_secret=secret,
)

print('GS Session initialized.')

## Step 2: Create a New Factor Risk Report

#### Already have a factor risk report?

<i>If you want to skip creating a new report and continue this tutorial with an existing factor risk report, run the following and skip to Step 3:</i>

In [2]:
risk_report_id = 'FACTOR RISK REPORT ID'

risk_report = FactorRiskReport.get(risk_report_id)

When creating a factor risk report, you must specify the risk model you would like to use.


If you would like to see all available risk model IDs to choose from, run the following:

In [ ]:
risk_models = FactorRiskModel.get_many()
for risk_model in risk_models:
    print(f'{risk_model.id}\n')

In this tutorial, we'll create a factor risk report leveraging the Barra USSLOW Long model. If you would like to calculate
risk in relation to a benchmark, you can add an index, basket, or ETF to your `FactorRiskReport` object:

In [ ]:
entity_id = 'PORTFOLIO ID'
risk_model_id = 'RISK MODEL ID'


risk_report = FactorRiskReport(
    risk_model_id=risk_model_id,
    fx_hedged=True,
)

risk_report.set_position_source(entity_id)
risk_report.save()

print(f'A new factor risk report for entity "{entity_id}" has been made with ID "{risk_report.id}".')

## Step 3: Schedule the Report

When scheduling reports, you have two options:
- Backcast the report: Take the earliest date with positions in the portfolio / basket and run the report on the positions held then with a start date before the earliest position date and an end date
 of the earliest position date
- Do not backcast the report: Set the start date as a date that has positions in the portfolio or basket and an end date after that (best practice is to set it to T-1). In this case the
 report will run on positions held as of each day in the date range

In this case, let's try scheduling the report without backcasting:

### Note:
Cannot Schedule backcasted report with no position sets on the report job end date

In [ ]:
start_date = dt.date(2025, 4, 1)
end_date = dt.date(2025, 4, 17)

risk_report.schedule(start_date=start_date, end_date=end_date, backcast=True)

print(f'Report "{risk_report.id}" has been scheduled.')

## Alternative Step 3: Run the Report

Depending on the size of your portfolio and the length of the schedule range, it usually takes anywhere from a couple seconds to a couple minutes for your report to finish executing.
Only after that can you successfully pull the results from that report. If you would rather run the report and pull the results immediately after they are ready, you can leverage the `run`
function.

You can run a report synchronously or asynchronously.
- Synchronous: the Python script will stall at the `run` function line and wait for the report to finish. The `run` function will then return a dataframe with the report results
- Asynchronously: the Python script will not stall at the `run` function line. The `run` function will return a `ReportJobFuture` object that will contain the report results when they are ready.

In this example, let's run the report asynchronously and wait for the results:

### Note:
Cannot Schedule historical report with no position sets before the report job start date

In [ ]:
start_date = dt.date(2025, 4, 2)
end_date = dt.date(2025, 4, 7)

report_result_future = risk_report.run(start_date=start_date, end_date=end_date, backcast=False, is_async=True)

while not report_result_future.done():
    print('Waiting for report results...')
    sleep(5)

print('\nReport results done! Here they are...')
print(report_result_future.result())

In [ ]:
start_date = dt.date(2025, 2, 6)
end_date = dt.date(2025, 3, 31)

report_result_future = risk_report.run(start_date=start_date, end_date=end_date, backcast=False, is_async=True)

while not report_result_future.done():
    print('Waiting for report results...')
    sleep(5)

print('\nReport results done! Here they are...')
print(report_result_future.result())

## Step 4: Pull Report Results

Now that we have our completed factor risk report, we can leverage the unique functionalities of the `FactorRiskReport` class to pull attribution and risk data. 

This `get_results` function will return the date indexed data as shown in the dummy table below:

|date      |factor  |factorCategory|exposure|pnl|dailyRisk|annualRisk|proportionOfRisk|relativeMarginalContributionToRisk|aum|marginalContributionToRiskPercent|
|----------|------------|---------------|-----------|--------------|--------------|------------|------------|-------------|----|-------|
|2025-04-10|	Specific|	Aggregations|	0.000000|	-186.921292|	116.350041|	1846.999635|	0.235141|	895.635113|	0.0|	NaN|
|2025-04-11|	Specific|	Aggregations|	0.000000|	-17.822047|	117.014467|	1857.547084|	0.230338|	891.503322|	0.0|	NaN|
|2025-04-14|	Specific|	Aggregations|	0.000000|	73.498315|	118.745143|	1885.020702|	0.225550|	895.235643|	0.0|	NaN|
|2025-04-15|	Specific|	Aggregations|	0.000000|	58.400802|	119.321131|	1894.164234|	0.223980|	896.442371|	2000000.0|	0.044822|
|2025-04-16|	Specific|	Aggregations|	0.000000|	23.749292|	116.631423|	1851.466441|	0.222957|	874.230618|	1000000.0|	0.087423|

In [ ]:
factor_and_total_results = risk_report.get_results(
    factors=['Factor', 'Specific'], start_date=dt.date(2025, 4, 10), end_date=dt.date(2025, 4, 17)
)
factor_and_total_results

### Quick Tip & Plotting!
You can pull historical data on factor, specific, and total PnL:

In [ ]:
pnl = risk_report.get_factor_pnl(factor_names=['Factor', 'Specific', 'Total'], start_date=start_date, end_date=end_date)
pnl.set_index('Date', inplace=True)
pnl.index = pd.to_datetime(pnl.index)

pnl.cumsum().plot(title='Risk Attribution Breakdown')

Now let's pull the breakdown of proportion of risk among the different factor types over time:

In [ ]:
prop_of_risk = risk_report.get_factor_proportion_of_risk(
    factor_names=['Market', 'Style', 'Industry', 'Country'], start_date=start_date, end_date=end_date
)
prop_of_risk.set_index('Date', inplace=True)
prop_of_risk.index = pd.to_datetime(prop_of_risk.index)

prop_of_risk.plot(title='Factor Proportion of Risk Breakdown')

**Please find more functions to pull specific factor risk data available [here](https://developer.gs.com/docs/gsquant/api/classes/gs_quant.markets.report.FactorRiskReport.html#gs_quant.markets.report.FactorRiskReport).**

### You're all set; Congrats!

*Other questions? Reach out to the [Portfolio Analytics team](mailto:gs-marquee-analytics-support@gs.com)!*
